In [2]:
import sys
import pandas as pd
import json
import ollama

def read_codex_data(path):
    df_train = pd.read_csv(f'{path}/train.txt', sep='\t', header=None, names=['Head','Relation','Tail'])
    df_test = pd.read_csv(f'{path}/test.txt', sep='\t', header=None, names=['Head','Relation','Tail'])
    df_valid = pd.read_csv(f'{path}/valid.txt', sep='\t', header=None, names=['Head','Relation','Tail'])

    with open('data/codex/entities.json', 'r') as file:
        entities = json.load(file)

    with open('data/codex/relations.json', 'r') as file:
        relations = json.load(file)
    return df_train, df_test, df_valid, entities, relations

In [3]:
def id2name_df(id_df:pd.DataFrame, entities_dict:dict, relations_dict:dict)-> pd.DataFrame:
    name_dict = {}
    for id, row in id_df.iterrows():
        # get id
        head_id = row['Head']
        relation_id = row['Relation']
        tail_id = row['Tail']
        # get label out of id
        head = entities_dict[head_id]['label']
        relation = relations_dict[relation_id]['label']
        tail = entities_dict[tail_id]['label']
        name_dict[id] = [head,relation,tail]
    name_df = pd.DataFrame.from_dict(name_dict,orient='index',columns=['Head','Relation','Tail'])
    return name_df

In [11]:
path = 'data/codex-m'
df_train, df_test, df_valid, entities, relations = read_codex_data(path)
df_train_name = id2name_df(df_train, entities, relations)
df_test_name = id2name_df(df_test, entities, relations)
df_valid_name = id2name_df(df_valid, entities, relations)
df_name = pd.concat([df_train_name, df_test_name, df_valid_name]).reset_index(drop=True)
df_name.to_csv(f'{path}/data.csv',index=False)
df_name.sample(60000).to_csv(f'{path}/data_sample.csv',index=False)
